In [7]:
import glob
import networkx as nx
import os
from karateclub import Graph2Vec
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import torch
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
import torch.nn as nn
from collections import defaultdict
import json
import os

In [8]:
l1_dir = "../v2/graphs"

In [9]:
all_graphs = {}

files_with_no_nodes = [
    '../v2/graphs/Technology/learnmachinelearning.json.graphml',
    '../v2/graphs/Technology/DataHoarder.json.graphml',
    '../v2/graphs/Technology/talesfromtechsupport.json.graphml',
    '../v2/graphs/Technology/technews.json.graphml',
    '../v2/graphs/Technology/apolloapp.json.graphml',
    '../v2/graphs/Technology/ipad.json.graphml',
    '../v2/graphs/Technology/onions.json.graphml',
    '../v2/graphs/Technology/Windows10.json.graphml',
    '../v2/graphs/Art/AnalogCommunity.json.graphml',
    '../v2/graphs/Art/iWallpaper.json.graphml',
    '../v2/graphs/Art/ImaginaryHorrors.json.graphml',
    '../v2/graphs/Art/pic.json.graphml',
    '../v2/graphs/Art/ArtHistory.json.graphml',
    '../v2/graphs/Art/80s.json.graphml',
    '../v2/graphs/Art/Pyrography.json.graphml',
    '../v2/graphs/Art/blenderhelp.json.graphml',
    '../v2/graphs/Art/MobileWallpaper.json.graphml',
    '../v2/graphs/Art/wallpaperengine.json.graphml'
]

# for each folder in l1_dir, get 100 arbitrary files
for folder in os.listdir(l1_dir):
    if folder == ".DS_Store":
        continue
    all_files = glob.glob(l1_dir + "/" + folder + "/*.graphml")
    # take the 100 smallest files
    all_files.sort(key=os.path.getsize)
    all_files = [f for f in all_files if f not in files_with_no_nodes]
    all_files = all_files
    all_graphs[folder] = all_files

for folder, files in all_graphs.items():
    print(folder, len(files))

Internet Culture and Memes 595
Music 303
Gaming 388
Television 260
Technology 398
Art 230
Funny 150
Business, Economics, and Finance 176
Sports 300
Animals and Pets 378
Learning and Education 190
Place 467
Food and Drink 197


In [5]:
from tqdm import tqdm
import networkx as nx
import time

In [6]:
def process_graph_with_timeout(file, label, graph_data, labels, total_files_per_label, processed_count, total_files, timeout=10):
    """
    Processes a single graph file with a timeout.
    Updates the processed count after processing.
    """
    start_time = time.time()
    graph = None
    
    try:
        while time.time() - start_time < timeout:
            # Attempt to load the graph
            graph = nx.to_undirected(nx.read_graphml(file))
            # reindex the graph
            mapping = {node: i for i, node in enumerate(graph.nodes())}
            graph = nx.relabel_nodes(graph, mapping)
            break  # Break the loop if successful
    except Exception as e:
        print(f"Error processing {file}: {e}")
        graph = None

    if graph is None or nx.is_empty(graph):
        print(f"Skipping {file} (empty or timeout)")
    else:
        graph_data.append(graph)
        labels.append(label)
        total_files_per_label[label] += 1
        print(f"Added {file}")
    
    processed_count[0] += 1  # Update the processed count
    print(f"Progress: {processed_count[0]}/{total_files} files processed.")
    
    return total_files_per_label[label] < 200  # Continue processing if limit not reached


def build_graph_from_lcc_with_graph2vec(all_graphs, timeout=10):
    """
    Processes multiple graph files with manual progress tracking.
    """
    graph_data = []  # To store graphs for Graph2Vec
    labels = []  # Labels for each graph
    total_files_per_label = {label: 0 for label in all_graphs.keys()}

    total_files = sum(len(files) for files in all_graphs.values())
    processed_count = [0]  # Use a mutable object to track processed files

    for folder, files in all_graphs.items():
        label = folder
        for file in files:
            if not process_graph_with_timeout(file, label, graph_data, labels, total_files_per_label, processed_count, total_files, timeout):
                print(f"Finished processing for label {label}")
                break

    return graph_data, labels


In [7]:
graph_data, labels = build_graph_from_lcc_with_graph2vec(all_graphs, timeout=120)

Added ../v2/graphs/Internet Culture and Memes/ThisLooksFun.json.graphml
Progress: 1/4032 files processed.
Added ../v2/graphs/Internet Culture and Memes/heartwarming.json.graphml
Progress: 2/4032 files processed.
Added ../v2/graphs/Internet Culture and Memes/ReactionMemes.json.graphml
Progress: 3/4032 files processed.
Added ../v2/graphs/Internet Culture and Memes/jag_ivl.json.graphml
Progress: 4/4032 files processed.
Added ../v2/graphs/Internet Culture and Memes/TwoSentenceHappiness.json.graphml
Progress: 5/4032 files processed.
Added ../v2/graphs/Internet Culture and Memes/fail.json.graphml
Progress: 6/4032 files processed.
Added ../v2/graphs/Internet Culture and Memes/meiaum.json.graphml
Progress: 7/4032 files processed.
Added ../v2/graphs/Internet Culture and Memes/CollegeMemes.json.graphml
Progress: 8/4032 files processed.
Added ../v2/graphs/Internet Culture and Memes/UFOdocumentaries.json.graphml
Progress: 9/4032 files processed.
Added ../v2/graphs/Internet Culture and Memes/nonono

In [8]:
model = Graph2Vec(dimensions=128)
model.fit(graph_data)

In [10]:
# Save the model
embedding = model.get_embedding()
embedding.shape

(2513, 128)

In [ ]:
label_encoder = LabelEncoder()

# Fit and transform the labels to numeric values
encoded_labels = label_encoder.fit_transform(labels)

# Print the original and encoded labels
for original, encoded in zip(labels, encoded_labels):
    print(f"Original: {original}, Encoded: {encoded}")

label_copy = labels.copy()
labels = encoded_labels

Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet Culture and Memes, Encoded: 6
Original: Internet C

In [29]:
np.save("graph2vec_embeddings.npy", embedding)
np.save("graph2vec_labels.npy", labels)

In [11]:
# load embeddings
embedding = np.load("graph2vec_embeddings.npy")
labels = np.load("graph2vec_labels.npy")

In [15]:
# Build similarity graph
def build_similarity_graph(embeddings, threshold):
    num_nodes = embeddings.shape[0]
    similarity_matrix = cosine_similarity(embeddings)

    G = nx.Graph()
    for i in range(num_nodes):
        G.add_node(i, embedding=embeddings[i])

    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if similarity_matrix[i, j] >= threshold:
                G.add_edge(i, j, weight=similarity_matrix[i, j])

    return G

# Convert to PyTorch Geometric data
def convert_to_torch_geometric(graph):
    node_features = np.array([graph.nodes[n]['embedding'] for n in graph.nodes])
    edge_index = np.array(list(graph.edges)).T
    edge_weights = np.array([graph[u][v]['weight'] for u, v in graph.edges])

    x = torch.tensor(node_features, dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    edge_weights = torch.tensor(edge_weights, dtype=torch.float)

    data = Data(x=x, edge_index=edge_index, edge_attr=edge_weights)

    return data

class NodeClassificationGNN(nn.Module):
    def __init__(self, input_dim, hidden_channels, num_classes, dropout=0.5):
        super().__init__()
        self.conv1 = SAGEConv(input_dim, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.dropout = nn.Dropout(dropout)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, num_classes)
        )

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)
        x = self.conv2(x, edge_index).relu()
        x = self.dropout(x)
        x = self.mlp(x)
        return x

# Train-Test-Val Splits
def split_data(data, labels, train_ratio=0.7, val_ratio=0.2):
    train_idx, temp_idx, train_labels, temp_labels = train_test_split(
        np.arange(len(labels)), labels, test_size=(1 - train_ratio), stratify=labels
    )
    val_size = val_ratio / (1 - train_ratio)
    val_idx, test_idx, val_labels, test_labels = train_test_split(
        temp_idx, temp_labels, test_size=(1 - val_size), stratify=temp_labels
    )
    return train_idx, val_idx, test_idx

def load_best_model(model, threshold, save_path='checkpoints'):
    checkpoint = torch.load(os.path.join(save_path, f'best_model_threshold_{threshold:.2f}.pt'))
    model.load_state_dict(checkpoint['model_state_dict'])
    return checkpoint['best_val_loss']

def save_training_history(history, threshold, save_path='training_history'):
    os.makedirs(save_path, exist_ok=True)
    with open(os.path.join(save_path, f'history_threshold_{threshold:.2f}.json'), 'w') as f:
        json.dump(history, f, indent=4)

class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False


def train(optimizer, criterion, model, data, _, threshold, save_path='checkpoints', num_epochs=300):

    # early_stopper = EarlyStopper(patience=100, min_delta=10)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                         factor=0.5, patience=10, 
                                                         min_lr=1e-6, verbose=True)
    
    history = defaultdict(list)
    os.makedirs(save_path, exist_ok=True)
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Calculate training metrics
        pred = out.argmax(dim=1)
        correct = int(pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
        train_acc = correct / data.train_mask.sum().item()

        # Validation
        model.eval()
        with torch.no_grad():
            val_out = model(data.x, data.edge_index)
            val_loss = criterion(val_out[data.val_mask], data.y[data.val_mask])
            val_pred = val_out.argmax(dim=1)
            val_correct = int(val_pred[data.val_mask].eq(data.y[data.val_mask]).sum().item())
            val_acc = val_correct / data.val_mask.sum().item()

        # Update learning rate
        scheduler.step(val_loss)

        # Save metrics
        history['train_loss'].append(loss.item())
        history['val_loss'].append(val_loss.item())
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        
        # Save best model
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_loss': best_val_loss
            }, os.path.join(save_path, f'best_model_threshold_{threshold:.2f}.pt'))
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, "
                  f"Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
            
        # if early_stopper.early_stop(val_loss.item()):             
        #     print(f"Early stopping at epoch {epoch}")
        #     break

    # Save training history
    save_training_history(history, threshold)
    
    print(f"Best Validation Loss: {best_val_loss:.4f}")

    return best_val_loss, history


def build_graphs_and_setup_model(embedding, thresholds=np.arange(0.1, 0.9, 0.1)):
    best_threshold = None
    best_test_acc = 0
    all_results = {}
    
    for threshold in thresholds:
        print(f"\nTraining with threshold: {threshold}")
        similarity_graph = build_similarity_graph(embedding, threshold=threshold)
        data = convert_to_torch_geometric(similarity_graph)
        data.y = torch.tensor(labels, dtype=torch.long)

        train_idx, val_idx, test_idx = split_data(data, labels)

        # Prepare masks
        train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

        train_mask[train_idx] = True
        val_mask[val_idx] = True
        test_mask[test_idx] = True

        data.train_mask = train_mask
        data.val_mask = val_mask
        data.test_mask = test_mask

        # Initialize model
        model = NodeClassificationGNN(input_dim=128, hidden_channels=64, num_classes=len(np.unique(labels)))
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        criterion = torch.nn.CrossEntropyLoss()

        # Train model
        _, history = train(optimizer, criterion, model, data, labels, threshold)
        model.eval()
        with torch.no_grad():
            test_out = model(data.x, data.edge_index)
            test_pred = test_out.argmax(dim=1)
            test_correct = int(test_pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
            test_acc = test_correct / data.test_mask.sum().item()

        print(f"Test Accuracy: {test_acc:.4f}")
        
        all_results[threshold] = {
            'test_accuracy': test_acc,
            'training_history': history
        }
        
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_threshold = threshold

    if not os.path.exists('results'):
        os.makedirs('results')
    # Save overall results
    with open('results/overall_results.json', 'w') as f:
        json.dump({
            'best_threshold': best_threshold,
            'best_test_accuracy': best_test_acc,
            'all_thresholds_results': {str(k): v for k, v in all_results.items()}
        }, f, indent=4)

    print(f"\nBest Threshold: {best_threshold:.2f}, Best Test Accuracy: {best_test_acc:.4f}")

if __name__ == "__main__":
    build_graphs_and_setup_model(embedding)



Training with threshold: 0.1


/opt/homebrew/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Loss: 2.5742, Val Loss: 2.5521, Acc: 0.0836, Val Acc: 0.1215
Epoch 10, Loss: 2.0269, Val Loss: 1.8320, Acc: 0.2581, Val Acc: 0.3327
Epoch 20, Loss: 1.6483, Val Loss: 1.3965, Acc: 0.3695, Val Acc: 0.4960
Epoch 30, Loss: 1.3209, Val Loss: 1.1108, Acc: 0.4844, Val Acc: 0.6275
Epoch 40, Loss: 1.2231, Val Loss: 1.0250, Acc: 0.4969, Val Acc: 0.6195
Epoch 50, Loss: 1.0997, Val Loss: 0.8128, Acc: 0.5435, Val Acc: 0.7012
Epoch 60, Loss: 0.9779, Val Loss: 0.7817, Acc: 0.5708, Val Acc: 0.7072
Epoch 70, Loss: 0.9564, Val Loss: 0.8179, Acc: 0.6072, Val Acc: 0.6892
Epoch 80, Loss: 0.8956, Val Loss: 0.6850, Acc: 0.6339, Val Acc: 0.7390
Epoch 90, Loss: 0.8184, Val Loss: 0.6337, Acc: 0.6549, Val Acc: 0.7789
Epoch 100, Loss: 0.8083, Val Loss: 0.6024, Acc: 0.6458, Val Acc: 0.7829
Epoch 110, Loss: 0.7510, Val Loss: 0.6479, Acc: 0.6669, Val Acc: 0.7450
Epoch 120, Loss: 0.7541, Val Loss: 0.6277, Acc: 0.6833, Val Acc: 0.7510
Epoch 130, Loss: 0.7425, Val Loss: 0.6565, Acc: 0.6947, Val Acc: 0.7371
Epo

/opt/homebrew/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Loss: 2.5735, Val Loss: 2.5550, Acc: 0.0807, Val Acc: 0.1454
Epoch 10, Loss: 2.2138, Val Loss: 2.0518, Acc: 0.1939, Val Acc: 0.2649
Epoch 20, Loss: 1.7654, Val Loss: 1.5187, Acc: 0.3286, Val Acc: 0.4701
Epoch 30, Loss: 1.5257, Val Loss: 1.3898, Acc: 0.3860, Val Acc: 0.4880
Epoch 40, Loss: 1.3024, Val Loss: 1.1742, Acc: 0.4849, Val Acc: 0.5518
Epoch 50, Loss: 1.1581, Val Loss: 0.9416, Acc: 0.5179, Val Acc: 0.6235
Epoch 60, Loss: 1.1142, Val Loss: 0.9296, Acc: 0.5384, Val Acc: 0.6534
Epoch 70, Loss: 1.1201, Val Loss: 0.9014, Acc: 0.5270, Val Acc: 0.6633
Epoch 80, Loss: 1.0435, Val Loss: 0.8607, Acc: 0.5657, Val Acc: 0.6454
Epoch 90, Loss: 0.9208, Val Loss: 0.8343, Acc: 0.6066, Val Acc: 0.6594
Epoch 100, Loss: 0.9875, Val Loss: 0.7383, Acc: 0.5998, Val Acc: 0.7470
Epoch 110, Loss: 0.8434, Val Loss: 0.7040, Acc: 0.6475, Val Acc: 0.7331
Epoch 120, Loss: 0.8489, Val Loss: 0.6904, Acc: 0.6396, Val Acc: 0.7271
Epoch 130, Loss: 0.8070, Val Loss: 0.6405, Acc: 0.6515, Val Acc: 0.7669
Epo

...

Epoch 0, Loss: 2.5773, Val Loss: 2.5560, Acc: 0.0716, Val Acc: 0.0817
Epoch 10, Loss: 2.3980, Val Loss: 2.3583, Acc: 0.1870, Val Acc: 0.2072
Epoch 20, Loss: 2.1303, Val Loss: 2.0865, Acc: 0.2723, Val Acc: 0.2928
Epoch 30, Loss: 1.8487, Val Loss: 1.9838, Acc: 0.3519, Val Acc: 0.3406
Epoch 40, Loss: 1.6548, Val Loss: 1.9504, Acc: 0.4099, Val Acc: 0.3307
Epoch 50, Loss: 1.4982, Val Loss: 1.9946, Acc: 0.4576, Val Acc: 0.3586
Epoch 60, Loss: 1.4415, Val Loss: 2.0079, Acc: 0.4878, Val Acc: 0.3606
Epoch 70, Loss: 1.3806, Val Loss: 2.0342, Acc: 0.4991, Val Acc: 0.3645
Epoch 80, Loss: 1.3481, Val Loss: 2.0597, Acc: 0.5099, Val Acc: 0.3725
Epoch 90, Loss: 1.3119, Val Loss: 2.0773, Acc: 0.5230, Val Acc: 0.3685
Epoch 100, Loss: 1.2887, Val Loss: 2.1009, Acc: 0.5310, Val Acc: 0.3546
Epoch 110, Loss: 1.2914, Val Loss: 2.0993, Acc: 0.5179, Val Acc: 0.3566
Epoch 120, Loss: 1.2818, Val Loss: 2.0962, Acc: 0.5333, Val Acc: 0.3606
Epoch 130, Loss: 1.2538, Val Loss: 2.0973, Acc: 0.5333, Val Acc: 0.3625
Epo

/opt/homebrew/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Loss: 2.5755, Val Loss: 2.5585, Acc: 0.0739, Val Acc: 0.1016
Epoch 10, Loss: 2.3838, Val Loss: 2.2810, Acc: 0.1887, Val Acc: 0.2351
Epoch 20, Loss: 2.1108, Val Loss: 2.0969, Acc: 0.2763, Val Acc: 0.2948
Epoch 30, Loss: 1.8892, Val Loss: 1.9974, Acc: 0.3508, Val Acc: 0.3586
Epoch 40, Loss: 1.7370, Val Loss: 1.9315, Acc: 0.3894, Val Acc: 0.3606
Epoch 50, Loss: 1.5974, Val Loss: 1.9549, Acc: 0.4434, Val Acc: 0.3725
Epoch 60, Loss: 1.4901, Val Loss: 1.9906, Acc: 0.4730, Val Acc: 0.3825
Epoch 70, Loss: 1.4240, Val Loss: 2.0324, Acc: 0.5014, Val Acc: 0.3586
Epoch 80, Loss: 1.3592, Val Loss: 2.0522, Acc: 0.5173, Val Acc: 0.3845
Epoch 90, Loss: 1.3656, Val Loss: 2.0797, Acc: 0.5122, Val Acc: 0.3884
Epoch 100, Loss: 1.3354, Val Loss: 2.0869, Acc: 0.5264, Val Acc: 0.3845
Epoch 110, Loss: 1.3550, Val Loss: 2.0919, Acc: 0.5162, Val Acc: 0.3785
Epoch 120, Loss: 1.3525, Val Loss: 2.0937, Acc: 0.5281, Val Acc: 0.3805
Epoch 130, Loss: 1.3372, Val Loss: 2.0932, Acc: 0.5139, Val Acc: 0.3805
Epo

/opt/homebrew/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Loss: 2.5673, Val Loss: 2.5563, Acc: 0.0836, Val Acc: 0.0757
Epoch 10, Loss: 2.3517, Val Loss: 2.3296, Acc: 0.1859, Val Acc: 0.2291
Epoch 20, Loss: 2.1501, Val Loss: 2.1398, Acc: 0.2564, Val Acc: 0.3008
Epoch 30, Loss: 1.9561, Val Loss: 2.0901, Acc: 0.3229, Val Acc: 0.2888
Epoch 40, Loss: 1.7372, Val Loss: 2.0870, Acc: 0.3917, Val Acc: 0.3247
Epoch 50, Loss: 1.6240, Val Loss: 2.0449, Acc: 0.4349, Val Acc: 0.3486
Epoch 60, Loss: 1.5297, Val Loss: 2.0675, Acc: 0.4684, Val Acc: 0.3586
Epoch 70, Loss: 1.4296, Val Loss: 2.1330, Acc: 0.5094, Val Acc: 0.3406
Epoch 80, Loss: 1.4019, Val Loss: 2.1743, Acc: 0.5202, Val Acc: 0.3546
Epoch 90, Loss: 1.3923, Val Loss: 2.2116, Acc: 0.5162, Val Acc: 0.3606
Epoch 100, Loss: 1.3067, Val Loss: 2.2214, Acc: 0.5503, Val Acc: 0.3546
Epoch 110, Loss: 1.3202, Val Loss: 2.2336, Acc: 0.5406, Val Acc: 0.3546
Epoch 120, Loss: 1.3187, Val Loss: 2.2462, Acc: 0.5429, Val Acc: 0.3625
Epoch 130, Loss: 1.3147, Val Loss: 2.2503, Acc: 0.5463, Val Acc: 0.3625
Epo